In [ ]:
import pyaudio

p = pyaudio.PyAudio()

stream = p.open(
    format=pyaudio.paInt16,  # 16-bit PCM
    channels=2,              # stereo
    rate=48000,              # 48kHz
    output=True
)

import asyncio
from google import genai
from google.genai import types

api_key = "API-KEY"
client = genai.Client(
    api_key = api_key,
    http_options = {"api_version": "v1alpha"}
)

async def main():
      async def receive_audio(session):

        # streaming audio to PC/default audio
        async for message in session.receive():
          if message.server_content and message.server_content.audio_chunks:
            audio_data = message.server_content.audio_chunks[0].data
            stream.write(audio_data)

      async with (
        client.aio.live.music.connect(model = 'models/lyria-realtime-exp') as session,
        asyncio.TaskGroup() as tg,
      ):
        
        await session.set_weighted_prompts(
          prompts=[(

            # Four different genres with the weight of the user selected genre being higher
            types.WeightedPrompt(text = 'Salsa', weight = 1.0)),
            types.WeightedPrompt(text = 'Afrobeat', weight = 15.0),
            types.WeightedPrompt(text = 'Rock', weight = 1.0),
            types.WeightedPrompt(text = 'Romance', weight = 1.0),

            # Genre with sensor input from temp. sensor
            types.WeightedPrompt(text = 'Summer Heat', weight = 14.0)
          ])
        await session.set_music_generation_config(
          # photo-resister -> brightness (tonal quality), heart-rate -> variability of prompt, walking pace -> tempo (bpm)
          config=types.LiveMusicGenerationConfig(brightness = 0.9, temperature = 1.0, bpm = 130))
        
        tg.create_task(receive_audio(session))

        await session.play()

await main()

CancelledError: 